In [1]:
# try to load a model

from transformers import AutoModelForCausalLM, AutoTokenizer
import yaml
import torch
# from coconut.coconut import Coconut
from coconut.utils import Config, set_seed, ProgressCallbackNoPrint, rm_old_prog_cb


## Load

In [3]:
from coconut.coconut import (
    CoconutConfig,
    CoconutQwen2ForCausalLM,
)

f = "../outputs/gsm-qwen-1.5b_20250130-193831/checkpoint_1"
f2 = f + '/config.yaml'
with open(fc) as f2:
    config_dict = yaml.safe_load(f2)
configs = Config(config_dict)

model = CoconutQwen2ForCausalLM.from_pretrained(f)
tokenizer = AutoTokenizer.from_pretrained(f)
# model.load_state_dict(safe_open(f1, 'pt'))

## Eval

In [5]:
from coconut.dataset import (
    CoconutCollator,
    get_cot_latent_dataset,
    get_dataset,
    get_question_only_latent_dataset,
)
from coconut.eval import evaluate

bot_id = tokenizer.convert_tokens_to_ids("<|start-latent|>")
eot_id = tokenizer.convert_tokens_to_ids("<|end-latent|>")
scheduled_stage = 3
latent_id = tokenizer.convert_tokens_to_ids("<|latent|>")

max_size = 1024
base_dataset_valid = get_dataset(
    configs.val_path, tokenizer, max_size=max_size//30+3, drop_unused=False
)

dataset_gen_val = get_question_only_latent_dataset(
    scheduled_stage,
    base_dataset_valid,
    configs,
    bot_id,
    latent_id,
    eot_id,
    no_bot_eot=False,
    # drop_unused=False,
)
collator = CoconutCollator(tokenizer, latent_id=latent_id, label_pad_token_id=-100)
valid_gen_dataloader = torch.utils.data.DataLoader(
    dataset_gen_val,
    num_workers=1,
    pin_memory=True,
    batch_size=1,
    collate_fn=collator,
)

# run eval
max_new_tokens = 64
device = "cuda"
dtype = torch.bfloat16
phase = -1

r = evaluate(valid_gen_dataloader, model, tokenizer, base_dataset_valid, max_new_tokens=max_new_tokens, name=f"eval_{phase}", dtype=dtype, device=device)

Qwen2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "../outputs/gsm-qwen-1.5b_20250130-193831/checkpoint_1",
  "architectures": [
    "CoconutQwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "latent_token_id": 151667,
  "max_position_embeddings": 4096,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "replacement_method": "0.5",
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.2",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151668
}